In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/02/25 11:00:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [47]:
FILE = '../../de_bootcamp/week5/data/raw/fhvhv/2021/02/fhvhv_tripdata_2021-02.csv'
TARGET_DIR= '../../de_bootcamp/week5/data/parquet/'

In [10]:
from pyspark.sql import types

In [45]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [48]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv(FILE)

In [49]:
df = df.repartition(24)

In [50]:
df.write.parquet(f'{TARGET_DIR}test', mode='overwrite')

In [53]:
df = spark.read.parquet(f'{TARGET_DIR}test')

In [54]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02887|2021-02-06 01:18:35|2021-02-06 01:40:34|         163|         235|   null|
|           HV0005|              B02510|2021-02-05 07:13:06|2021-02-05 07:31:56|         225|         181|   null|
|           HV0003|              B02869|2021-02-04 16:56:52|2021-02-04 17:21:36|         260|          95|   null|
|           HV0003|              B02871|2021-02-03 18:34:17|2021-02-03 18:57:12|         235|          60|   null|
|           HV0003|              B02869|2021-02-04 07:25:09|2021-02-04 07:30:34|          55|          55|   null|
|           HV0003|              B02836|2021-02-04 23:15:27|2021-02-04 23:34:29|

In [18]:
df.groupBy('dispatching_base_num').count().orderBy('count', ascending = False).show()  

+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
|              B02869| 429720|
|              B02887| 322331|
|              B02871| 312364|
|              B02864| 311603|
|              B02866| 311089|
|              B02878| 305185|
|              B02682| 303255|
|              B02617| 274510|
|              B02883| 251617|
|              B02884| 244963|
|              B02882| 232173|
|              B02876| 215693|
|              B02879| 210137|
|              B02867| 200530|
|              B02877| 198938|
+--------------------+-------+
only showing top 20 rows



In [24]:
z_file = '../../de_bootcamp/week5/data/taxi+_zone_lookup.csv'
target_zones =  '../../de_bootcamp/week5/data/parquet/zones'

In [25]:
zone_schema = types.StructType([
    types.StructField('LocationID', types.IntegerType(), True),
    types.StructField('Borough', types.StringType(), True),
    types.StructField('Zone', types.StringType(), True),
    types.StructField('service_zone', types.StringType(), True)
])

In [28]:
df_zones = spark.read.option("header", "true").schema(zone_schema).csv(z_file)

In [29]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [30]:
df_zones.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [31]:
df_zones= df_zones.repartition(24)

In [32]:
df_zones = df_zones.write.parquet(target_zones)

In [34]:
df_z = spark.read.parquet(target_zones)

In [35]:
df_z.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|       127|    Manhattan|              Inwood|   Boro Zone|
|       206|Staten Island|Saint George/New ...|   Boro Zone|
|        62|     Brooklyn| Crown Heights South|   Boro Zone|
|       186|    Manhattan|Penn Station/Madi...| Yellow Zone|
|       244|    Manhattan|Washington Height...|   Boro Zone|
|       110|Staten Island|    Great Kills Park|   Boro Zone|
|       122|       Queens|              Hollis|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|        16|       Queens|             Bayside|   Boro Zone|
|       238|    Manhattan|Upper West Side N...| Yellow Zone|
|        77|     Brooklyn|East New York/Pen...|   Boro Zone|
|       131|       Queens|     Jamaica Estates|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|       190|     Brookly

In [36]:
df_z.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [60]:
from pyspark.sql.functions import concat,col, concat_ws

In [63]:
df2=df.select(concat_ws('/',df.PULocationID,df.DOLocationID).alias("pair_loc"))

In [64]:
df2.show()

+--------+
|pair_loc|
+--------+
| 163/235|
| 225/181|
|  260/95|
|  235/60|
|   55/55|
|   74/81|
| 239/231|
|  95/196|
| 241/265|
|  90/137|
|     7/7|
| 231/246|
|  108/29|
|  11/112|
| 229/234|
|  10/218|
|  49/232|
|  125/17|
|   47/68|
|  41/223|
+--------+
only showing top 20 rows



In [67]:
df.sort('PULocationID', 'DOLocationID').show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02875|2021-02-14 06:46:37|2021-02-14 06:49:47|           1|           1|   null|
|           HV0004|              B02800|2021-02-09 19:37:11|2021-02-09 20:10:00|           1|           4|   null|
|           HV0004|              B02800|2021-02-27 17:45:56|2021-02-27 18:31:31|           1|           4|   null|
|           HV0004|              B02800|2021-02-20 15:23:25|2021-02-20 16:00:14|           1|           4|   null|
|           HV0004|              B02800|2021-02-05 08:41:49|2021-02-05 09:17:19|           1|          14|   null|
|           HV0004|              B02800|2021-02-28 21:20:25|2021-02-28 21:45:57|

In [71]:
df.groupBy('PULocationID' ).count().orderBy('count', ascending = False).show()   

+------------+------+
|PULocationID| count|
+------------+------+
|          61|203777|
|          76|166959|
|          37|140636|
|          79|137901|
|          42|137246|
|          17|125394|
|         225|120026|
|          39|117751|
|         188|117734|
|          89|117092|
|         244|115590|
|           7|110787|
|         231|109074|
|          74|103440|
|         181|103267|
|          35|103059|
|         234| 96469|
|         170| 94307|
|         249| 93355|
|          75| 91517|
+------------+------+
only showing top 20 rows



In [72]:
df_result = df.join(df_z, df.PULocationID == df_z.LocationID)

In [73]:
df_result.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+----------+---------+--------------------+------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|LocationID|  Borough|                Zone|service_zone|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+----------+---------+--------------------+------------+
|           HV0003|              B02887|2021-02-06 01:18:35|2021-02-06 01:40:34|         163|         235|   null|       163|Manhattan|       Midtown North| Yellow Zone|
|           HV0005|              B02510|2021-02-05 07:13:06|2021-02-05 07:31:56|         225|         181|   null|       225| Brooklyn|  Stuyvesant Heights|   Boro Zone|
|           HV0003|              B02869|2021-02-04 16:56:52|2021-02-04 17:21:36|         260|          95|   null|       260|   Queens|            Woo

In [74]:
df_result = df_result.join(df_z, df_result.DOLocationID == df_z.LocationID)

In [82]:
df.registerTempTable('joined')

In [87]:
df_z.registerTempTable('zones')

In [88]:
df_result=spark.sql("""
SELECT 
*
FROM zones

""").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|       127|    Manhattan|              Inwood|   Boro Zone|
|       206|Staten Island|Saint George/New ...|   Boro Zone|
|        62|     Brooklyn| Crown Heights South|   Boro Zone|
|       186|    Manhattan|Penn Station/Madi...| Yellow Zone|
|       244|    Manhattan|Washington Height...|   Boro Zone|
|       110|Staten Island|    Great Kills Park|   Boro Zone|
|       122|       Queens|              Hollis|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|        16|       Queens|             Bayside|   Boro Zone|
|       238|    Manhattan|Upper West Side N...| Yellow Zone|
|        77|     Brooklyn|East New York/Pen...|   Boro Zone|
|       131|       Queens|     Jamaica Estates|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|       190|     Brookly

In [92]:
df_result = spark.sql("""
SELECT 
t.PULocationID, t.DOLocationID,  p_z.Borough AS puBorough, p_z.Zone AS puZone, d_z.Borough AS doBorough , d_z.Zone AS doZone
FROM joined AS t
JOIN zones AS p_z ON p_z.LocationID = t.PULocationID
JOIN zones AS d_z ON p_z.LocationID = t.DOLocationID
""")

In [96]:
df_result.registerTempTable('result')

In [105]:
df_res2 = spark.sql("""
SELECT 
puZone, doZone,
count(*) AS cnt
FROM result 
GROUP BY  puZone, doZone
ORDER BY cnt DESC
""")

In [106]:
df_res2.show(truncate=False)

+-------------------+---------------------------------------------+------+
|puZone             |doZone                                       |cnt   |
+-------------------+---------------------------------------------+------+
|East New York      |Governor's Island/Ellis Island/Liberty Island|135123|
|Borough Park       |Governor's Island/Ellis Island/Liberty Island|111987|
|East New York      |Corona                                       |90082 |
|Canarsie           |Governor's Island/Ellis Island/Liberty Island|84078 |
|Crown Heights North|Governor's Island/Ellis Island/Liberty Island|77928 |
|Borough Park       |Corona                                       |74658 |
|Canarsie           |Corona                                       |56052 |
|Bay Ridge          |Governor's Island/Ellis Island/Liberty Island|53802 |
|Crown Heights North|Corona                                       |51952 |
|East New York      |Greenwich Village North                      |45041 |
|East New York      |Spri